In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df=pd.read_excel('/content/classifier new.xlsx')
df.columns = df.columns.str.replace(' ', '')
df.head()

,Sentences,OpinionTerm1,AspectCategory1,AspectPolarity1,OpinionTerm2,AspectCategory2,Aspectpolarity2
0,excellent service....very professional electri...,excellent service,Company_Service,positive,professional electricians,service_person_attitude,positive
1,nice and good app,good app,App_Utility,positive,NaN,NaN,NaN
2,came on time work proeffetionaly complete,on time,service_schedule,positive,work proeffetionaly,service_person_attitude,positive
3,first time user and very satisfied.,first time user,App_Functionality,positive,satisfied,Company_Service,positive
4,nice facilities and rates is to much high,nice facilities,Company_Service,positive,much high,cost,negative


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [ ]:
# df['Aspect_Category1'].value_counts()
selected_columns = ['Sentences', 'OpinionTerm1','AspectCategory1','AspectPolarity1']
df_data = df[selected_columns]

In [ ]:
df.columns

Index(['Sentences', 'OpinionTerm1', 'AspectCategory1', 'AspectPolarity1',
       'OpinionTerm2', 'AspectCategory2', 'Aspectpolarity2'],
      dtype='object')

In [ ]:
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
nltk.download('punkt')
nltk.download('stopwords')
# Preprocessing
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)


def preprocess_text(text):
    text = str(text)
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords and token not in punctuation]
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df_data = df_data.copy()
df_data['Sentences' ] = df_data['Sentences'].apply(preprocess_text)
df_data['OpinionTerm1' ] = df_data[ 'OpinionTerm1'].apply(preprocess_text)
df_data['AspectCategory1' ] = df_data[ 'AspectCategory1'].apply(preprocess_text)

In [ ]:
#Xfeatures = df_data[['Sentences', 'OpinionTerm1']]
Xfeatures = df_data[['OpinionTerm1','AspectCategory1']]
#ylabels = df_data[['AspectCategory1']]
ylabels = df_data[['AspectPolarity1']]

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(Xfeatures,ylabels,test_size=0.3,random_state=7)

In [ ]:
# Random Forest clasifier
Xfeatures["combined"] = Xfeatures["Sentences"]+ " " + Xfeatures['OpinionTerm1']
X = Xfeatures["combined"]
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

<ipython-input-15-9c9fed37db60>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xfeatures["combined"] = Xfeatures["Sentences"]+ " " + Xfeatures['OpinionTerm1']


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

dict_classifiers = {
    "SVM":svm.SVC(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "Naive B":  MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),

}
data = df = pd.DataFrame(columns = ['model', 'Accuracy_AC1'])
for model, model_instantiation in dict_classifiers.items():
  clf = MultiOutputClassifier(model_instantiation)
  clf.fit(X_train_vec, y_train)
  y_pred = clf.predict(X_test_vec)
  AC_1 = accuracy_score(y_test["AspectCategory1"].values, y_pred)
  data.loc[len(data)] = [model, AC_1]
  conf_matrix = confusion_matrix(y_test["AspectCategory1"].values, y_pred)
  print(classification_report(y_test["AspectCategory1"].values, y_pred, zero_division=0))



                                                                      precision    recall  f1-score   support

                                                   App_Functionality       1.00      0.31      0.47        13
                                                              App_UI       0.83      0.59      0.69        17
                                                         App_Utility       0.88      0.90      0.89       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.73      0.95      0.83       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.53      0.57      0.55        14
         

In [ ]:
data

,model,Accuracy_AC1
0,SVM,0.749100
1,KNN,0.701080
2,Naive B,0.650660
3,XGBoost,0.687875


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

dict_classifiers = {
    "SVM": svm.SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Naive B": MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
}
data = pd.DataFrame(columns=['model', 'Precision_AC1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    precision_AC1 = precision_score(y_test['AspectCategory1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, precision_AC1]



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

In [ ]:
data

,model,Precision_AC1
0,SVM,0.369932
1,KNN,0.306901
2,Naive B,0.154322
3,XGBoost,0.276058


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

dict_classifiers = {
    "SVM": svm.SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Naive B": MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
}
data = pd.DataFrame(columns=['model', 'Recall_AC1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    recall_AC1 = recall_score(y_test['AspectCategory1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, recall_AC1]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
data

,model,Recall_AC1
0,SVM,0.254867
1,KNN,0.282432
2,Naive B,0.090392
3,XGBoost,0.215956


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

dict_classifiers = {
    "SVM": svm.SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Naive B": MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
}
data = pd.DataFrame(columns=['model', 'F1_Score_AC1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    f1_score_AC1 = f1_score(y_test['AspectCategory1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, f1_score_AC1]

In [ ]:
data

,model,F1_Score_AC1
0,SVM,0.282069
1,KNN,0.291495
2,Naive B,0.090216
3,XGBoost,0.231925


In [ ]:
ylabels = df_data[['AspectPolarity1']]
Xfeatures["combined"] =  Xfeatures['OpinionTerm1']+ " " + Xfeatures['AspectCategory1']
X = Xfeatures["combined"]
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

<ipython-input-16-47a93e848d8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xfeatures["combined"] =  Xfeatures['OpinionTerm1']+ " " + Xfeatures['AspectCategory1']


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

dict_classifiers = {
    "SVM":svm.SVC(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "Naive B":  MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),

}
data = df = pd.DataFrame(columns = ['model', 'Accuracy_AP1'])
for model, model_instantiation in dict_classifiers.items():
  clf = MultiOutputClassifier(model_instantiation)
  clf.fit(X_train_vec, y_train)
  y_pred = clf.predict(X_test_vec)
  AP_1 = accuracy_score(y_test["AspectPolarity1"].values, y_pred)
  data.loc[len(data)] = [model, AP_1]
  conf_matrix = confusion_matrix(y_test["AspectPolarity1"].values, y_pred)
  print(classification_report(y_test["AspectPolarity1"].values, y_pred, zero_division=0))


              precision    recall  f1-score   support

    Positive       0.56      0.75      0.64       319
   negative        0.84      0.82      0.83       180
    positive       0.59      0.41      0.49       324
   positive        0.00      0.00      0.00        10

    accuracy                           0.62       833
   macro avg       0.50      0.49      0.49       833
weighted avg       0.62      0.62      0.61       833

              precision    recall  f1-score   support

    Positive       0.51      0.61      0.56       319
   negative        0.83      0.76      0.79       180
    positive       0.53      0.47      0.50       324
   positive        0.00      0.00      0.00        10

    accuracy                           0.58       833
   macro avg       0.47      0.46      0.46       833
weighted avg       0.58      0.58      0.58       833

              precision    recall  f1-score   support

    Positive       0.55      0.71      0.62       319
   negative        0.

In [ ]:
data

,model,Accuracy_AP1
0,SVM,0.624250
1,KNN,0.581032
2,Naive B,0.612245
3,XGBoost,0.602641


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
data = pd.DataFrame(columns=['model', 'Precision_AP1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    precision_AC1 = precision_score(y_test['AspectPolarity1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, precision_AC1]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
data

,model,Precision_AC1
0,SVM,0.495014
1,KNN,0.467145
2,Naive B,0.486713
3,XGBoost,0.477429


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
data = pd.DataFrame(columns=['model', 'Recall_AP1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    precision_AC1 = precision_score(y_test['AspectPolarity1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, precision_AC1]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
data

,model,Recall_AP1
0,SVM,0.495014
1,KNN,0.467145
2,Naive B,0.486713
3,XGBoost,0.477429


In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
data = pd.DataFrame(columns=['model', 'F1_Score_AP1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    precision_AC1 = precision_score(y_test['AspectPolarity1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, precision_AC1]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
data

,model,F1_Score_AP1
0,SVM,0.495014
1,KNN,0.467145
2,Naive B,0.486713
3,XGBoost,0.477429
